In [48]:
from scipy.stats import norm

In [49]:
import yfinance as yf, math, pandas as pd, numpy as np

In [50]:
from datetime import datetime

In [51]:
def cdf(x):
    return 0.5*(1+math.erf(x/math.sqrt(2)))
            

In [52]:
def d1(S,K, r, t, vol):
    return (math.log(S/K)+(r + (vol**2)/2)*t)/(vol*math.sqrt(t))

In [53]:
def d2(S, K, r, t, vol):
    return d1(S, K, r, t, vol) - vol*math.sqrt(t)

In [54]:
def call(S, K, r, t, vol):
    Nd1 = cdf(d1(S, K, r, t, vol))
    Nd2= cdf(d2(S, K, r, t, vol))
    return S*Nd1-K*math.exp(-r*t)*Nd2

In [55]:
def put(S, K, r, t, vol):
    Nd1 = cdf(-d1(S, K, r, t, vol))
    Nd2 = cdf(-d2(S, K, r, t, vol))
    return K*math.exp(-r*t)*Nd2 - S*Nd1

In [56]:
def optiondata(symbol, date):
    option=yf.Ticker(symbol)
    options_data=option.option_chain(date)
    return options_data.calls, options_data.puts

In [57]:
def getS(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    return stock.history(period="1d")['Close'].iloc[-1]

In [58]:
def getrate():
    symbol = "^TNX"
    now=datetime.now()
    ago10 = now.replace(year=now.year-10)
    treasury_data = yf.download(symbol, start=ago10, end=now)
    last_yield = treasury_data['Close'].iloc[-1]
    return last_yield

In [59]:
def get_t(date_str):
    date_obj=datetime.strptime(date_str, '%Y-%m-%d')
    diff=(date_obj-datetime.now()).days/365.25
    return diff

In [60]:
def calculate_vol(symb):
    data = yf.download(symb, start=datetime.now().replace(year=datetime.now().year-10), end=datetime.now())
    data['Daily_Return'] = data['Adj Close'].pct_change()
    daily_vol = data['Daily_Return'].std()
    annual_vol=daily_vol*252**0.5
    return annual_vol
    

In [61]:
def delta_call(S, k, r, t, vol):
    return norm.cdf(d1(S, k, r, t, vol))

In [62]:
def delta_put(S, k, r, t, vol):
    return norm.cdf(d1(S, k, r, t, vol))-1

In [63]:
def gamma_call(S, k, r, t, vol):
    return norm.pdf(d1(S, k, r, t, vol))/(S*vol*np.sqrt(t))

In [64]:
def gamma_put(S, k, r, t, vol):
    return norm.pdf(d1(S, k, r, t, vol))/(S*vol*np.sqrt(t))

In [65]:
def vega(S, k, r, t, vol):
    return S*np.sqrt(t)*norm.pdf(d1(S, k, r, t, vol))

In [66]:
def rho_call(S, k, r, t, vol):
    return k*t*S*norm.cdf(d2(S, k, r, t, vol))
def rho_put(S, k, r, t, vol):
    return -k*t*S*norm.cdf(-d2(S, k, r, t, vol))

In [67]:
def theta_call(S, k, r, t, vol):
    return -(S*norm.pdf(d1(S, k, r, t, vol))*vol/(2*np.sqrt(t)))-r*k*np.exp(-r*t)*norm.cdf(d2(S, k, r, t, vol))
def theta_put(S, k, r, t, vol):
    return -(S*norm.pdf(d1(S, k, r, t, vol))*vol/(2*np.sqrt(t)))+r*k*np.exp(-r*t)*norm.cdf(-d2(S, k, r, t, vol))

In [68]:
symbol ='AAPL'
S= getS(symbol)
vol=calculate_vol(symbol)
r=getrate()/100
vol=calculate_vol(symbol)
option = yf.Ticker(symbol)
print(f"Risk free rate {r}\nCurrent Stock Price {S}\nVolatility {vol}")
combined = pd.DataFrame()
for i in range(len(option.options)):
    date1=option.options[i]
    calls, puts = optiondata(symbol, date1)
    t=get_t(date1)
    col_drop=['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
    calls.drop(columns=col_drop, inplace=True)
    calls['bsmValuation'] = calls.apply(lambda row: call(S, row['strike'], t, r, vol), axis=1)
    calls['diff']=calls.apply(lambda row: (row['bsmValuation']-row['ask']), axis=1)
    calls['delta']=calls.apply(lambda row: delta_call(S, row['strike'], t, r, vol), axis=1)
    calls['gamma']=calls.apply(lambda row: gamma_call(S, row['strike'], t, r, vol), axis=1)
    calls['vega']=calls.apply(lambda row: vega(S, row['strike'], t, r, vol), axis=1)
    calls['rho']=calls.apply(lambda row: rho_call(S, row['strike'], t, r, vol), axis =1)
    calls['theta']=calls.apply(lambda row: theta_call(S, row['strike'], t,r,vol), axis=1)
    combined=pd.concat([combined, calls], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Risk free rate 0.044630002975463864
Current Stock Price 194.19000244140625
Volatility 0.2831469735255448


In [69]:
combined

,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,bsmValuation,diff,delta,gamma,vega,rho,theta
0,AAPL240607C00100000,100.0,92.55,93.75,0.0,0.0,0.000010,True,9.422665e+01,0.476653,1.000000e+00,4.281862e-29,2.040447e-26,8.666700e+02,-8.210542e-01
1,AAPL240607C00110000,110.0,82.35,83.45,0.0,0.0,0.000010,True,8.423032e+01,0.780318,1.000000e+00,6.065031e-22,2.890185e-19,9.533370e+02,-9.031596e-01
2,AAPL240607C00120000,120.0,72.30,73.30,0.0,0.0,0.000010,True,7.423398e+01,0.933983,1.000000e+00,2.231559e-16,1.063410e-13,1.040004e+03,-9.852650e-01
3,AAPL240607C00125000,125.0,67.30,68.50,0.0,0.0,0.000010,True,6.923582e+01,0.735815,1.000000e+00,4.396783e-14,2.095210e-11,1.083338e+03,-1.026318e+00
4,AAPL240607C00130000,130.0,62.40,63.80,0.0,0.0,0.000010,True,6.423765e+01,0.437648,1.000000e+00,4.540845e-12,2.163861e-09,1.126671e+03,-1.067370e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,AAPL261218C00310000,310.0,5.65,6.30,0.0,0.0,0.257362,False,3.452379e-09,-6.300000,1.863200e-09,9.715365e-10,4.629688e-07,3.478316e-06,-2.378130e-06
931,AAPL261218C00320000,320.0,4.35,5.55,0.0,0.0,0.258476,False,1.117872e-10,-5.550000,6.521696e-11,3.691797e-11,1.759261e-08,1.218382e-07,-8.766517e-08
932,AAPL261218C00330000,330.0,3.80,4.75,0.0,0.0,0.257423,False,3.103404e-12,-4.750000,1.945375e-12,1.185680e-12,5.650150e-10,3.636551e-09,-2.743213e-09
933,AAPL261218C00340000,340.0,3.20,3.95,0.0,0.0,0.254646,False,7.618329e-14,-3.950000,5.027993e-14,3.276757e-14,1.561481e-11,9.403837e-11,-7.412208e-11


In [70]:
combined.to_csv('combined_data.csv', index=False)